In [ ]:
import os
import time
import pickle
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

%matplotlib inline

In [ ]:
video_dir = '/mnt/disks/datastorage/videos/'
weights_dir = '/mnt/disks/datastorage/weights/'

# Video stream
video_file = 'cold_day/demos/shooting_ballroom_light6.mov'
video_path = os.path.join(video_dir, 'fens', video_file)

# For live stream
# video_path = './stream.sdp'

# Parameters
nb_classes = 3
skip_num = 1
svm_model = os.path.join(weights_dir, 'svm_demo.pkl')
prediction_path = '/mnt/disks/datastorage/predictions/'
flownet_model = 'FlowNet2CSS'

# Models
flownet_weights = os.path.join(weights_dir, 'FlowNet2-CSS.pth.tar')
spatial_weights = os.path.join(weights_dir, 'spatial_weights_demo.pth.tar')
motion_weights = os.path.join(weights_dir, 'motion_weights_demo.pth.tar')

In [ ]:
# Run pipeline and ignore warnings
!python3 -W ignore pipeline.py --stream $video_path \
                               --nb_classes $nb_classes \
                               --skip_frames $skip_num \
                               --svm $svm_model \
                               --save $prediction_path \
                               --model $flownet_model \
                               --optical_weights $flownet_weights \
                               --spatial_weights $spatial_weights \
                               --motion_weights $motion_weights

In [ ]:
prediction_files = sorted(glob.glob(os.path.join(prediction_path, 'predictions_*.pkl')), reverse=True)

with open(prediction_files[0], 'rb') as pf:
    predictions = pickle.load(pf)
    predictions = np.array(predictions).squeeze()
    actions = np.argmax(predictions, axis=1)
    
predictions_argmax = predictions.argmax(1)
for pred in list(zip(predictions, predictions_argmax)):
    print(pred)

In [ ]:
# Interpolate predictions for skipped frames
predictions_extended = np.zeros((skip_num + 1) * len(predictions_argmax))
for i in range(len(predictions_extended)):
    predictions_extended[i] = int(predictions_argmax[i // (skip_num + 1)])

In [ ]:
action_type = 'demo'
action_dict = {
    'fens': {
        0: 'Theft',
        1: 'Assault',
        2: 'Shooting',
        3: 'No Action',
    },
    'demo': {
        0: 'Assault',
        1: 'Shooting',
        2: 'No Action',
    }
}

cap = cv2.VideoCapture(video_path)

frame_time = 0
frame_idx = 0
display = None
i = 0
while True:
    start_time = time.time()
    
    ret, frame = cap.read()
    if not ret:
        break
    
    try:
        label = action_dict[action_type][actions[frame_idx]]
    except IndexError:
        break
        
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), thickness=3)
    
    plt.axis('off')
    plt.imshow(frame)
    plt.show()
    clear_output(wait=True)

    frame_time += time.time() - start_time
    frame_idx += 1
    
cap.release()

frame_time /= frame_idx
print('Avg. frame processing time: {} s'.format(round(frame_time, 2)))